Goal of analysis script is to pull in excel files with manually entered intake and weight data, determine daily intake per body weight (for both water and ethanol) for subsequent analysis


In [1]:
#getting and working with data
import pandas as pd
import scipy.stats as stats
import numpy as np
import re
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

In [2]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.directory = filedialog.askdirectory()
print (root.directory)

C:/Users/Schindler/Documents/Schindler_Lab/Data/Ethanol/Intermit 2BC/2BC 6dq r2/mice


In [3]:
path_names = []
files = os.listdir(root.directory)
for file in files: 
        path_names.append(root.directory + "/" + file)

path_names[0]

'C:/Users/Schindler/Documents/Schindler_Lab/Data/Ethanol/Intermit 2BC/2BC 6dq r2/mice/645_1m.xlsx'

In [4]:
group_path = "C:/Users/Schindler/Documents/Schindler_Lab/Data/Ethanol/Intermit 2BC/2BC 6dq r2/groups.xlsx"
data = pd.read_excel(group_path)
df_mice = pd.DataFrame(data = data)
df_mice.head()

,Animal,Cage_#,Treatment,Number,Group
0,645,205,1,3,4
1,646,205,1,3,4
2,648,206,1,3,4
3,649,206,1,3,4
4,650,206,1,3,4


Loop through path names to create pandas data table from excel file. Get animal number from path name (animal number is file name), create array of animal number the length of the data_table and add this as a column.

In [5]:
data_start = pd.DataFrame()

for path in path_names:
    data_excel = pd.read_excel(path)
    data_table_int = pd.DataFrame(data = data_excel)
    
    name = path
    name = re.search("\d\d\d", path).group(0)
    name_long = [name] * len(data_table_int)
    
    group = df_mice.loc[df_mice['Animal'] == int(name), 'Group'].item()
    group_long = [group] * len(data_table_int)
    
    data_table_int = data_table_int.assign(Animal = name_long)
    data_table_int = data_table_int.assign(Group = group_long)  
    
    data_start = data_start.append(data_table_int, ignore_index=True)


data_start.head()

,Day,Day #,EtOH,Weight,EtOH bf,EtOH after,H2O b4,H2O after,H2O b4.1,H2O after.1,EtOH 8,H20 8,Animal,Group
0,Friday,1,0,26.9,x,x,90.3,81.7,89.1,42.2,x,x,645,4
1,Sunday,2,6,26.9,74.5,71.7,81.7,80.2,x,x,x,x,645,4
2,Monday,3,6,26.9,71.7,69,80.2,78.5,x,x,x,x,645,4
3,Tuesday,4,6,26.9,69,66,78.5,77.1,x,x,x,x,645,4
4,Wednesday,5,6,26.9,66,62.7,77.1,75.8,x,x,65,76.7,645,4


In [6]:
data_start.replace('x', 0, inplace=True)

In [7]:
def C2BC(data_start):
    "Analyzes two bottle choice data"
    
    #H20 24h data
    data_start['H2O_intake_1_raw'] = data_start['H2O b4'] - data_start['H2O after']
    
    data_start['H2O_intake_2_raw'] = data_start['H2O b4.1'] - data_start['H2O after.1']
    
    data_start['H2O_intake_raw'] = data_start['H2O_intake_1_raw'] + data_start['H2O_intake_2_raw']

    data_start['H2O_intake_24'] = data_start['H2O_intake_raw'] / (data_start['Weight'] /1000)
    
    #H20 8pm data
    if data_start['H20 8'] > 0:
        data_start['H2O_intake_raw_8'] = data_start['H2O b4'] - data_start['H20 8']

        data_start['H2O_intake_8'] = data_start['H2O_intake_raw_8'] / (data_start['Weight'] /1000)
    else:
        data_start['H2O_intake_raw_8'] = np.nan
        data_start['H2O_intake_8'] = np.nan

    #EtOH 24h data
    data_start['EtOH_intake_raw'] = data_start['EtOH bf'] - data_start['EtOH after']

    data_start['EtOH_intake'] = (data_start['EtOH_intake_raw'] * data_start['EtOH'] / 100) / (data_start['Weight'] /1000)

    #EtOH 8pm data
    if data_start['EtOH 8'] > 0:
        data_start['EtOH_intake_raw_8'] = data_start['EtOH bf'] - data_start['EtOH 8']

        data_start['EtOH_intake_8'] = (data_start['EtOH_intake_raw_8'] * data_start['EtOH'] / 100) / (data_start['Weight'] /1000)
    else:
        data_start['EtOH_intake_raw_8'] = np.nan
        data_start['EtOH_intake_8'] = np.nan
        
    #preference data
    data_start['EtOH_pref'] = data_start['EtOH_intake_raw'] / (data_start['EtOH_intake_raw'] + data_start['H2O_intake_raw'])

    if data_start['EtOH 8'] > 0:
        data_start['EtOH_pref_8'] = \
        data_start['EtOH_intake_raw_8'] / (data_start['EtOH_intake_raw_8'] + data_start['H2O_intake_raw_8'])
    else:
        data_start['EtOH_pref_8'] = np.nan
    
    return data_start

In [8]:
data_calc = data_start.apply(C2BC, axis = 1)
data_calc.head(9)

,Day,Day #,EtOH,Weight,EtOH bf,EtOH after,H2O b4,H2O after,H2O b4.1,H2O after.1,EtOH 8,H20 8,Animal,Group,H2O_intake_1_raw,H2O_intake_2_raw,H2O_intake_raw,H2O_intake_24,H2O_intake_raw_8,H2O_intake_8,EtOH_intake_raw,EtOH_intake,EtOH_intake_raw_8,EtOH_intake_8,EtOH_pref,EtOH_pref_8
0,Friday,1,0,26.9,0.0,0.0,90.3,81.7,89.1,42.2,0.0,0.0,645,4,8.6,46.9,55.5,2063.197026,NaN,NaN,0.0,0.000000,NaN,NaN,0.000000,NaN
1,Sunday,2,6,26.9,74.5,71.7,81.7,80.2,0.0,0.0,0.0,0.0,645,4,1.5,0.0,1.5,55.762082,NaN,NaN,2.8,6.245353,NaN,NaN,0.651163,NaN
2,Monday,3,6,26.9,71.7,69.0,80.2,78.5,0.0,0.0,0.0,0.0,645,4,1.7,0.0,1.7,63.197026,NaN,NaN,2.7,6.022305,NaN,NaN,0.613636,NaN
3,Tuesday,4,6,26.9,69.0,66.0,78.5,77.1,0.0,0.0,0.0,0.0,645,4,1.4,0.0,1.4,52.044610,NaN,NaN,3.0,6.691450,NaN,NaN,0.681818,NaN
4,Wednesday,5,6,26.9,66.0,62.7,77.1,75.8,0.0,0.0,65.0,76.7,645,4,1.3,0.0,1.3,48.327138,0.4,14.869888,3.3,7.360595,1.0,2.230483,0.717391,0.714286
5,Thursday,6,6,26.9,62.7,59.7,75.8,74.6,0.0,0.0,0.0,0.0,645,4,1.2,0.0,1.2,44.609665,NaN,NaN,3.0,6.691450,NaN,NaN,0.714286,NaN
6,Friday,7,0,26.9,0.0,0.0,74.6,71.0,81.1,70.4,0.0,0.0,645,4,3.6,10.7,14.3,531.598513,NaN,NaN,0.0,0.000000,NaN,NaN,0.000000,NaN
7,Tuesday,11,6,26.9,65.0,62.1,71.0,69.8,0.0,0.0,0.0,0.0,645,4,1.2,0.0,1.2,44.609665,NaN,NaN,2.9,6.468401,NaN,NaN,0.707317,NaN
8,Wednesday,12,6,26.9,65.8,64.5,69.8,66.6,0.0,0.0,0.0,0.0,645,4,3.2,0.0,3.2,118.959108,NaN,NaN,1.3,2.899628,NaN,NaN,0.288889,NaN


In [9]:
data_calc_only = data_calc[['Animal', 'Group', 'Day #', 'EtOH', 'H2O_intake_24', 'EtOH_intake', 'EtOH_pref']]
data_calc_only.head(9)

,Animal,Group,Day #,EtOH,H2O_intake_24,EtOH_intake,EtOH_pref
0,645,4,1,0,2063.197026,0.000000,0.000000
1,645,4,2,6,55.762082,6.245353,0.651163
2,645,4,3,6,63.197026,6.022305,0.613636
3,645,4,4,6,52.044610,6.691450,0.681818
4,645,4,5,6,48.327138,7.360595,0.717391
5,645,4,6,6,44.609665,6.691450,0.714286
6,645,4,7,0,531.598513,0.000000,0.000000
7,645,4,11,6,44.609665,6.468401,0.707317
8,645,4,12,6,118.959108,2.899628,0.288889


In [37]:
#process etoh intake (ave over first six days), etoh deprivation (day 11), and quinine (day 12) data
EtOH_intake = data_calc_only.loc[(data_calc_only['Day #'] > 0) & (data_calc_only['Day #'] < 7)].groupby('Animal').mean()
EtOH_intake.columns = [str(col) + '_6dave' for col in EtOH_intake.columns]

EtOH_dep_intake = data_calc_only.loc[data_calc_only['Day #'] == 11].groupby('Animal').mean()
EtOH_dep_intake.columns = [str(col) + '_dep' for col in EtOH_dep_intake.columns]

EtOH_dep_intake['H2O_intake_dep_diff'] = EtOH_dep_intake['H2O_intake_24_dep'] / EtOH_intake['H2O_intake_24_6dave']
EtOH_dep_intake['EtOH_intake_dep_diff'] = EtOH_dep_intake['EtOH_intake_dep'] / EtOH_intake['EtOH_intake_6dave']
EtOH_dep_intake['EtOH_pref_dep_diff'] = EtOH_dep_intake['EtOH_pref_dep'] / EtOH_intake['EtOH_pref_6dave']

EtOH_quinine_intake = data_calc_only.loc[data_calc_only['Day #'] == 12].groupby('Animal').mean()
EtOH_quinine_intake.columns = [str(col) + '_quin' for col in EtOH_quinine_intake.columns]

EtOH_quinine_intake['H2O_intake_quin_diff'] = EtOH_quinine_intake['H2O_intake_24_quin'] / EtOH_intake['H2O_intake_24_6dave']
EtOH_quinine_intake['EtOH_intake_quin_diff'] = EtOH_quinine_intake['EtOH_intake_quin'] / EtOH_intake['EtOH_intake_6dave']
EtOH_quinine_intake['EtOH_pref_quin_diff'] = EtOH_quinine_intake['EtOH_pref_quin'] / EtOH_intake['EtOH_pref_6dave']

In [38]:
EtOH_final = pd.DataFrame()
EtOH_final['Group'] = EtOH_intake['Group_6dave']
EtOH_final = pd.concat([EtOH_final, EtOH_intake.loc[:, 'H2O_intake_24_6dave':]], axis = 1)
EtOH_final = pd.concat([EtOH_final, EtOH_dep_intake.loc[:, 'H2O_intake_24_dep':]], axis = 1)
EtOH_final = pd.concat([EtOH_final, EtOH_quinine_intake.loc[:, 'H2O_intake_24_quin':]], axis = 1)
EtOH_final.head()

,Group,H2O_intake_24_6dave,EtOH_intake_6dave,EtOH_pref_6dave,H2O_intake_24_dep,EtOH_intake_dep,EtOH_pref_dep,H2O_intake_dep_diff,EtOH_intake_dep_diff,EtOH_pref_dep_diff,H2O_intake_24_quin,EtOH_intake_quin,EtOH_pref_quin,H2O_intake_quin_diff,EtOH_intake_quin_diff,EtOH_pref_quin_diff
Animal,,,,,,,,,,,,,,,,
645,4.0,387.856258,5.501859,0.563049,44.609665,6.468401,0.707317,0.115016,1.175676,1.256226,118.959108,2.899628,0.288889,0.306709,0.527027,0.513079
646,4.0,122.529644,12.252964,0.670571,67.193676,8.063241,0.666667,0.548387,0.658065,0.994177,126.482213,2.134387,0.219512,1.032258,0.174194,0.327351
648,4.0,161.835749,4.094203,0.366682,57.971014,8.260870,0.703704,0.358209,2.017699,1.919109,134.057971,2.173913,0.212766,0.828358,0.530973,0.580246
649,4.0,100.891862,5.250836,0.571194,46.822742,8.628763,0.754386,0.464088,1.643312,1.320717,110.367893,1.605351,0.195122,1.093923,0.305732,0.341604
650,4.0,107.701564,5.054152,0.574788,61.371841,6.064982,0.622222,0.569832,1.200000,1.082525,90.252708,2.815884,0.342105,0.837989,0.557143,0.595185


In [39]:
EtOH_final.groupby('Group').mean()

,H2O_intake_24_6dave,EtOH_intake_6dave,EtOH_pref_6dave,H2O_intake_24_dep,EtOH_intake_dep,EtOH_pref_dep,H2O_intake_dep_diff,EtOH_intake_dep_diff,EtOH_pref_dep_diff,H2O_intake_24_quin,EtOH_intake_quin,EtOH_pref_quin,H2O_intake_quin_diff,EtOH_intake_quin_diff,EtOH_pref_quin_diff
Group,,,,,,,,,,,,,,,
4.0,176.163015,6.430803,0.549257,55.593788,7.497251,0.690859,0.411107,1.338950,1.314551,116.023979,2.325833,0.251679,0.819847,0.419014,0.471493
5.0,194.981832,3.928915,0.371794,71.296636,5.993331,0.571975,0.400838,1.584935,1.630109,120.245144,1.602656,0.180620,0.771171,0.431545,0.526014
